# Import libraries

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
# pio.templates.default = "plotly_white"

# Load the csv file with results into a pandas dataframe

In [ ]:
df = pd.read_csv("../results/broad_sweep.csv")
# df = pd.read_csv("../results/regularizing_sweep.csv")
# df = pd.read_csv("../results/jk_pooling_sweep.csv")
# df = pd.read_csv("../results/relaxed.csv")
df.head()

# Draw boxplots to compare hyperparameter settings

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

hyperparameters = ["architecture", "gnn_layers", "mlp_layers", "hidden_channels", "jk", "pool", "activation", "norm", "transform", "dropout", "selected_features"]
metrics_to_plot = ["good_within.99", "good_within.95"]

df = df.fillna(value="none")

if "selected_features" in df.columns:
    df["selected_features"] = df["selected_features"].apply(lambda x: "original" if "core_number" in x else "new")

for hyperparameter in hyperparameters:
    # Create a temporary dataframe for plotting, dropping rows where the hyperparameter is null
    plot_df = df

    # Filter out some values
    # plot_df = plot_df[~plot_df["pool"].isin(["median"])]
    # # plot_df = plot_df[plot_df["gnn_layers"] == 5]
    # plot_df = plot_df[plot_df["hidden_channels"] == 64]

    # Get the actual categories present in the data
    actual_categories = sorted(plot_df[hyperparameter].unique())

    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(f"Metrics vs {hyperparameter}", f"Duration vs {hyperparameter}"),
        horizontal_spacing=0.1
    )

    # Add main metrics to left subplot
    for i, metric in enumerate(metrics_to_plot):
        fig.add_trace(
            go.Box(x=plot_df[hyperparameter], y=plot_df[metric],
                   name=metric, boxmean=True,
                   marker_color=px.colors.qualitative.Plotly[i],
                   offsetgroup=str(i)),
            row=1, col=1
        )

    # Add duration metric to right subplot
    fig.add_trace(
        go.Box(x=plot_df[hyperparameter], y=plot_df['duration'],
               name='duration', boxmean=True,
               marker_color=px.colors.qualitative.Plotly[2],
               showlegend=False,
               offsetgroup='duration',
               width=0.6),
        row=1, col=2
    )

    # Update layout for both subplots
    fig.update_xaxes(
        type='category',
        categoryorder='array',
        categoryarray=actual_categories,
        row=1, col=1
    )
    fig.update_xaxes(
        type='category',
        categoryorder='array',
        categoryarray=actual_categories,
        row=1, col=2
    )

    fig.update_layout(
        title=f"Analysis of {hyperparameter}",
        height=400,
        boxmode='group',
        boxgap=0.2,
        boxgroupgap=0.1
    )
    fig.show()

# Best model for specified hyperparameter value

In [ ]:
hyperparameter = "selected_features"
metric = "good_within.99"

best_runs = df.loc[df.groupby(hyperparameter)[metric].idxmax()]
best_runs.head()

### Statistical significance of hyperparameter settings (t-test and ANOVA)

In [ ]:
import itertools
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import ttest_rel

# ------------------------------
# 0. Prepare data
# ------------------------------
stat_df = df.rename(columns={"good_within.99": "Score"})
# List of columns to remove (keep only hyperparameters + Score)
stat_df = stat_df[hyperparameters + ["Score"]]
unique_columns = [col for col in stat_df.columns if stat_df[col].nunique() == 1]
stat_df = stat_df.drop(columns=unique_columns)


In [ ]:
# -----------------------------
# 1. Paired comparison for X=a vs X=b
# -----------------------------
def paired_effect(df, param, val1, val2):
    # All other params except target and current param
    others = [c for c in df.columns if c not in [param, "Score"]]

    # Pivot: rows = all other params, cols = values of the chosen param
    pivot = df.pivot_table(index=others, columns=param, values="Score")

    # Only keep rows where both val1 and val2 exist
    if val1 not in pivot.columns or val2 not in pivot.columns:
        return None  # comparison not possible
    subset = pivot.dropna(subset=[val1, val2])
    if subset.empty:
        return None

    # Differences
    diffs = subset[val1] - subset[val2]
    mean_diff = diffs.mean()

    # Paired t-test
    tstat, pval = ttest_rel(subset[val1], subset[val2])

    return {
        "param": param,
        "val1": val1,
        "val2": val2,
        "mean_diff": mean_diff,
        "pval": pval,
        "n_pairs": len(diffs)
    }

# -----------------------------
# Loop through all hyperparams and all pairs of values
# -----------------------------
def all_paired_effects(df):
    results = []
    params = [c for c in df.columns if c != "Score"]

    for param in params:
        values = df[param].unique()
        for val1, val2 in itertools.combinations(values, 2):
            res = paired_effect(df, param, val1, val2)
            if res is not None:
                results.append(res)

    return pd.DataFrame(results)

results_df = all_paired_effects(stat_df)
results_df = results_df.sort_values(by="mean_diff", ascending=False, key=lambda col: col.abs())
print(results_df)

In [ ]:
# -----------------------------
# 2. Factorial ANOVA / Linear Model
# -----------------------------
params = [c for c in stat_df.columns if c != "Score"]
# Build a formula: Score ~ X * Y * Z
# This includes main effects + all interactions
# formula = "Score ~ " + " * ".join(params)

# Build a formula: Score ~ X:Y + X:Z + Y:Z
# This includes main effects + pairwise interactions
terms = params[:]  # main effects
terms += [f"{a}:{b}" for a, b in itertools.combinations(params, 2)]
formula = "Score ~ " + " + ".join(terms)

model = smf.ols(formula, data=stat_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # Type II ANOVA
anova_table = anova_table.sort_values(by="F", ascending=False)
print("\nANOVA results:")
print(anova_table)


In [ ]:
# -----------------------------
# 3. Interaction plots
# -----------------------------
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

def plot_interaction_grid(df, params, response="Score"):
    pairs = list(itertools.combinations(params, 2))
    n = len(pairs)
    ncols = 3  # how many plots per row
    nrows = -(-n // ncols)  # ceil division

    fig, axes = plt.subplots(nrows, ncols, figsize=(5*ncols, 4*nrows), squeeze=False)

    for ax, (p1, p2) in zip(axes.flatten(), pairs):
        sns.pointplot(
            data=df, x=p1, y=response, hue=p2,
            dodge=True, markers="o", capsize=0.1, errorbar="pi", ax=ax
        )
        ax.set_title(f"{p1} × {p2}")
        ax.legend(title=p2, fontsize="small", title_fontsize="small")

    # Remove empty subplots if any
    for ax in axes.flatten()[len(pairs):]:
        ax.remove()

    plt.tight_layout()
    plt.show()

# Example usage:
params = [c for c in stat_df.columns if c != "Score"]
plot_interaction_grid(stat_df, params)

In [ ]:
import os

def save_interaction_plot_pdf(df, factor1, factor2, response="Score", folder="plots"):
    os.makedirs(folder, exist_ok=True)
    plt.figure(figsize=(6,4))

    sns.pointplot(
        data=df, x=factor1, y=response, hue=factor2,
        dodge=True, markers="o", capsize=0.1, ci="sd",
    )

    plt.title(f"Interaction: {factor1} × {factor2}")

    filename = os.path.join(folder, f"{factor1}_x_{factor2}.pdf")
    plt.savefig(filename)
    plt.close()
    print(f"Saved plot to {filename}")

